In [0]:
use catalog uc_prod

### Getting the distinct states from the staing master table (silver.daily_pricing_silver)

In [0]:

create or replace table silver.dim_state_staging_st1_sc1
select distinct productgroup_name,product_name from silver.daily_pricing_silver
where record_updated_date > (
      select nvl(max(process_file_updated_date),'2010-01-01') from uc_prod.watermark_tbl.process_logs 
      where process_name = 'reporting_dim_product_gold_scd1' and process_status ='completed'
                            )


### Add the distinct unique id for the state items

In [0]:
create or replace table silver.dim_state_staging_st2
select 
st1.product_name,st1.productgroup_name,
gpr.PRODUCT_ID as gold_product_id,
case when gpr.PRODUCT_NAME is null 
  then row_number()over(order by st1.productgroup_name,st1.product_name) 
  else gpr.product_id end as rn_num
from silver.dim_state_staging_st1_sc1 as st1
left join gold.reporting_dim_product_gold as gpr
on lower(gpr.PRODUCT_NAME) = lower(st1.product_name)
where gpr.PRODUCT_NAME is null or st1.productgroup_name <> gpr.PRODUCTGROUP_NAME


In [0]:
create or replace table silver.dim_state_staging_st3
select 
case when t.gold_product_id is null 
  then (t.rn_num+s.max_flag) 
  else t.gold_product_id end as product_id,
t.product_name as product_name,
t.productgroup_name as productgroup_name
,current_timestamp() as lakehouse_inserted_date,
current_timestamp() as lakehouse_updated_date
from silver.dim_state_staging_st2 as t
cross join (select nvl(max(PRODUCT_ID),0) as max_flag from gold.reporting_dim_product_gold ) as s

In [0]:
merge into gold.reporting_dim_product_gold as tgt
using (
select * from silver.dim_state_staging_st3 as t
) as src
on tgt.PRODUCT_ID = src.product_id
when matched then update 
  set tgt.lakehouse_updated_date = current_timestamp(),
      tgt.PRODUCT_NAME  = src.PRODUCT_NAME,
      tgt.PRODUCTGROUP_NAME = src.productgroup_name
when not matched then  
  insert (
      PRODUCT_ID,
      PRODUCT_NAME,
      PRODUCTGROUP_NAME,
      lakehouse_inserted_date,
      lakehouse_updated_date
         )
  values
        (
        src.product_id,
        src.product_name,
        src.productgroup_name,
        src.lakehouse_inserted_date,
        src.lakehouse_updated_date
        )


### update the water mark table

In [0]:
insert into uc_prod.watermark_tbl.process_logs
(Process_name,Process_file_updated_date,process_status)
values
(
'reporting_dim_product_gold_scd1',
current_timestamp(),
'completed'
)

In [0]:
-- truncate table uc_prod.gold.reporting_dim_product_gold;
-- delete from uc_prod.watermark_tbl.process_logs where process_name = 'reporting_dim_product_gold_scd1'
